# Notebook 03 - Deploy e Uso do Modelo

**Projeto:** Previsão de Churn em Telecomunicações  
**Autores:** Pedro Dias, Gustavo Rodrigues  
**Data:** Dezembro 2025

---

## Objetivo

Demonstrar como:
1. Carregar o modelo treinado
2. Fazer predições para novos clientes
3. Interpretar os resultados
4. Criar uma função de predição reutilizável

In [ ]:
# ========== SETUP ==========
import pandas as pd
import numpy as np
import joblib
import warnings

warnings.filterwarnings('ignore')

print("Bibliotecas carregadas!")

## 1. Carregando o Modelo Salvo

In [ ]:
print("="*80)
print("CARREGANDO MODELO E ARTEFATOS")
print("="*80)

# Carregar modelo
modelo = joblib.load('modelo_final.pkl')
print("\nModelo carregado: modelo_final.pkl")
print(f"   Tipo: {type(modelo).__name__}")

# Carregar colunas de treino
feature_columns = joblib.load('feature_columns.pkl')
print("\nFeatures carregadas: feature_columns.pkl")
print(f"   Total de features: {len(feature_columns)}")

# Tentar carregar scaler (se existir)
try:
    scaler = joblib.load('scaler.pkl')
    print("\nScaler carregado: scaler.pkl")
    usa_scaler = True
except:
    print("\nScaler não encontrado (modelo não requer normalização)")
    scaler = None
    usa_scaler = False

print("\n" + "="*80)
print("MODELO PRONTO PARA USO!")
print("="*80)

## 2. Função de Predição Reutilizável

In [ ]:
def prever_churn(cliente_dict):
    """
    Prevê se um cliente vai dar churn.
    
    Parâmetros:
    -----------
    cliente_dict : dict
        Dicionário com as informações do cliente.
        Exemplo:
        {
            'tenure': 12,
            'MonthlyCharges': 75.50,
            'TotalCharges': 906.00,
            'Contract': 'Month-to-month',
            'InternetService': 'Fiber optic',
            'PaymentMethod': 'Electronic check',
            'OnlineSecurity': 'No',
            'TechSupport': 'No',
            'PaperlessBilling': 'Yes',
            'SeniorCitizen': 0
        }
    
    Retorna:
    --------
    dict com:
        - 'classe': 'Yes' ou 'No'
        - 'probabilidade_churn': float entre 0 e 1
        - 'risco': 'ALTO', 'MÉDIO' ou 'BAIXO'
        - 'recomendacao': string com ação sugerida
    """
    
    # 1. Criar DataFrame
    df_novo = pd.DataFrame([cliente_dict])
    
    # 2. Aplicar One-Hot Encoding
    df_encoded = pd.get_dummies(df_novo, drop_first=True)
    
    # 3. Garantir mesmas colunas do treino
    for col in feature_columns:
        if col not in df_encoded.columns:
            df_encoded[col] = 0
    df_encoded = df_encoded[feature_columns]
    
    # 4. Aplicar scaler se necessário
    if usa_scaler:
        df_encoded = scaler.transform(df_encoded)
    
    # 5. Fazer predição
    classe_predita = modelo.predict(df_encoded)[0]
    probabilidade_churn = modelo.predict_proba(df_encoded)[0][1]  # Prob da classe 'Yes'
    
    # 6. Determinar nível de risco
    if probabilidade_churn >= 0.7:
        risco = "ALTO"
        cor = "[ALTO RISCO]"
    elif probabilidade_churn >= 0.4:
        risco = "MÉDIO"
        cor = "[MÉDIO RISCO]"
    else:
        risco = "BAIXO"
        cor = "[BAIXO RISCO]"
    
    # 7. Gerar recomendação
    if probabilidade_churn >= 0.7:
        recomendacao = (
            "AÇÃO IMEDIATA NECESSÁRIA:\n"
            "  - Contato proativo da equipe de retenção\n"
            "  - Oferecer desconto de 25% por 6 meses\n"
            "  - Propor migração para contrato anual com benefícios\n"
            "  - Incluir serviços adicionais (TechSupport, OnlineSecurity) gratuitamente"
        )
    elif probabilidade_churn >= 0.4:
        recomendacao = (
            "MONITORAMENTO ATIVO:\n"
            "  - Incluir em campanha de engajamento\n"
            "  - Oferecer upgrade de plano com desconto de 15%\n"
            "  - Enviar pesquisa de satisfação"
        )
    else:
        recomendacao = (
            "MANUTENÇÃO:\n"
            "  - Cliente em situação estável\n"
            "  - Continuar comunicação regular\n"
            "  - Considerar para programa de indicação"
        )
    
    return {
        'classe': classe_predita,
        'probabilidade_churn': probabilidade_churn,
        'risco': risco,
        'cor': cor,
        'recomendacao': recomendacao
    }

print("Função prever_churn() definida!")

## 3. Exemplo 1: Cliente com ALTO Risco de Churn

In [ ]:
print("="*80)
print("EXEMPLO 1: CLIENTE COM ALTO RISCO")
print("="*80)

# Perfil de risco: cliente novo, contrato mensal, sem serviços adicionais
cliente_alto_risco = {
    'tenure': 3,  # Apenas 3 meses
    'MonthlyCharges': 85.50,  # Mensalidade alta
    'TotalCharges': 256.50,  # Total baixo (novo)
    'Contract': 'Month-to-month',  # Contrato mensal
    'InternetService': 'Fiber optic',
    'PaymentMethod': 'Electronic check',  # Método associado a alto churn
    'OnlineSecurity': 'No',  # Sem serviços extras
    'TechSupport': 'No',
    'PaperlessBilling': 'Yes',
    'SeniorCitizen': 0
}

print("\nPerfil do Cliente:")
for key, value in cliente_alto_risco.items():
    print(f"   {key:20s}: {value}")

# Fazer predição
resultado = prever_churn(cliente_alto_risco)

print("\n" + "="*80)
print("RESULTADO DA PREDIÇÃO")
print("="*80)
print(f"\n{resultado['cor']} Risco: {resultado['risco']}")
print(f"   Probabilidade de Churn: {resultado['probabilidade_churn']:.1%}")
print(f"   Classificação: {resultado['classe']}")
print(f"\n{resultado['recomendacao']}")

## 4. Exemplo 2: Cliente com BAIXO Risco de Churn

In [ ]:
print("\n" + "="*80)
print("EXEMPLO 2: CLIENTE COM BAIXO RISCO")
print("="*80)

# Perfil estável: cliente antigo, contrato longo, com serviços
cliente_baixo_risco = {
    'tenure': 48,  # 4 anos de contrato
    'MonthlyCharges': 65.00,
    'TotalCharges': 3120.00,  # Alto valor total
    'Contract': 'Two year',  # Contrato de 2 anos
    'InternetService': 'DSL',
    'PaymentMethod': 'Credit card (automatic)',
    'OnlineSecurity': 'Yes',  # Com serviços extras
    'TechSupport': 'Yes',
    'PaperlessBilling': 'No',
    'SeniorCitizen': 1
}

print("\nPerfil do Cliente:")
for key, value in cliente_baixo_risco.items():
    print(f"   {key:20s}: {value}")

# Fazer predição
resultado = prever_churn(cliente_baixo_risco)

print("\n" + "="*80)
print("RESULTADO DA PREDIÇÃO")
print("="*80)
print(f"\n{resultado['cor']} Risco: {resultado['risco']}")
print(f"   Probabilidade de Churn: {resultado['probabilidade_churn']:.1%}")
print(f"   Classificação: {resultado['classe']}")
print(f"\n{resultado['recomendacao']}")

## 5. Exemplo 3: Cliente com Risco MÉDIO

In [ ]:
print("\n" + "="*80)
print("EXEMPLO 3: CLIENTE COM RISCO MÉDIO")
print("="*80)

# Perfil intermediário: tempo razoável, mas sem compromisso longo
cliente_medio_risco = {
    'tenure': 18,  # 1 ano e meio
    'MonthlyCharges': 70.00,
    'TotalCharges': 1260.00,
    'Contract': 'One year',  # Contrato anual
    'InternetService': 'Fiber optic',
    'PaymentMethod': 'Bank transfer (automatic)',
    'OnlineSecurity': 'No',  # Sem alguns serviços
    'TechSupport': 'Yes',  # Tem suporte
    'PaperlessBilling': 'Yes',
    'SeniorCitizen': 0
}

print("\nPerfil do Cliente:")
for key, value in cliente_medio_risco.items():
    print(f"   {key:20s}: {value}")

# Fazer predição
resultado = prever_churn(cliente_medio_risco)

print("\n" + "="*80)
print("RESULTADO DA PREDIÇÃO")
print("="*80)
print(f"\n{resultado['cor']} Risco: {resultado['risco']}")
print(f"   Probabilidade de Churn: {resultado['probabilidade_churn']:.1%}")
print(f"   Classificação: {resultado['classe']}")
print(f"\n{resultado['recomendacao']}")

## 6. Análise em Lote (Múltiplos Clientes)

In [ ]:
print("\n" + "="*80)
print("ANÁLISE EM LOTE - MÚLTIPLOS CLIENTES")
print("="*80)

# Criar lista de clientes para análise
clientes_para_analise = [
    {'nome': 'Cliente A', **cliente_alto_risco},
    {'nome': 'Cliente B', **cliente_baixo_risco},
    {'nome': 'Cliente C', **cliente_medio_risco},
    {'nome': 'Cliente D', 'tenure': 6, 'MonthlyCharges': 90.0, 'TotalCharges': 540.0,
     'Contract': 'Month-to-month', 'InternetService': 'Fiber optic',
     'PaymentMethod': 'Electronic check', 'OnlineSecurity': 'No',
     'TechSupport': 'No', 'PaperlessBilling': 'Yes', 'SeniorCitizen': 0},
    {'nome': 'Cliente E', 'tenure': 36, 'MonthlyCharges': 55.0, 'TotalCharges': 1980.0,
     'Contract': 'Two year', 'InternetService': 'DSL',
     'PaymentMethod': 'Credit card (automatic)', 'OnlineSecurity': 'Yes',
     'TechSupport': 'Yes', 'PaperlessBilling': 'No', 'SeniorCitizen': 1},
]

# Analisar todos
resultados_lote = []

for cliente in clientes_para_analise:
    nome = cliente.pop('nome')
    resultado = prever_churn(cliente)
    resultados_lote.append({
        'Nome': nome,
        'Tenure': cliente['tenure'],
        'Mensalidade': cliente['MonthlyCharges'],
        'Contrato': cliente['Contract'],
        'Prob. Churn': f"{resultado['probabilidade_churn']:.1%}",
        'Risco': f"{resultado['cor']} {resultado['risco']}",
        'Predição': resultado['classe']
    })

# Criar DataFrame
df_resultados = pd.DataFrame(resultados_lote)

print("\nResultados:")
print(df_resultados.to_string(index=False))

# Estatísticas
print("\n" + "="*80)
print("ESTATÍSTICAS DO LOTE")
print("="*80)
print(f"\nTotal de clientes analisados: {len(clientes_para_analise)}")
print(f"Clientes com risco ALTO:  {sum(1 for r in resultados_lote if 'ALTO' in r['Risco'])}")
print(f"Clientes com risco MÉDIO: {sum(1 for r in resultados_lote if 'MÉDIO' in r['Risco'])}")
print(f"Clientes com risco BAIXO: {sum(1 for r in resultados_lote if 'BAIXO' in r['Risco'])}")

## 7. Simulação de Intervenção

In [ ]:
print("\n" + "="*80)
print("SIMULAÇÃO: E SE MODIFICARMOS O PERFIL DO CLIENTE?")
print("="*80)

# Cliente com alto risco
print("\nCENÁRIO INICIAL (Alto Risco):")
resultado_antes = prever_churn(cliente_alto_risco)
print(f"   Probabilidade de Churn: {resultado_antes['probabilidade_churn']:.1%}")

# Simular intervenção: migrar para contrato anual + adicionar serviços
cliente_apos_intervencao = cliente_alto_risco.copy()
cliente_apos_intervencao['Contract'] = 'One year'  # Migra para contrato anual
cliente_apos_intervencao['OnlineSecurity'] = 'Yes'  # Adiciona OnlineSecurity
cliente_apos_intervencao['TechSupport'] = 'Yes'  # Adiciona TechSupport

print("\nCENÁRIO APÓS INTERVENÇÃO:")
print("   Ações aplicadas:")
print("   - Migração para contrato anual")
print("   - Inclusão de OnlineSecurity gratuito")
print("   - Inclusão de TechSupport gratuito")

resultado_depois = prever_churn(cliente_apos_intervencao)
print(f"\n   Probabilidade de Churn: {resultado_depois['probabilidade_churn']:.1%}")

# Calcular impacto
reducao = (resultado_antes['probabilidade_churn'] - resultado_depois['probabilidade_churn']) * 100
print(f"\nREDUÇÃO NO RISCO: {reducao:.1f} pontos percentuais")

if resultado_depois['probabilidade_churn'] < 0.5:
    print("\nSUCESSO: Cliente saiu da zona de risco!")
else:
    print("\nCliente ainda em risco, mas com melhora significativa.")

## 8. Exportando Função para Produção

In [ ]:
# Criar arquivo Python com a função
codigo_producao = '''
"""\nFunção de predição de churn para uso em produção.\n
Uso:\n    from predicao_churn import prever_churn_cliente\n    resultado = prever_churn_cliente(dados_cliente)\n"""\n
import pandas as pd\nimport joblib\nimport warnings\nwarnings.filterwarnings('ignore')\n
# Carregar artefatos\nmodelo = joblib.load('modelo_final.pkl')\nfeature_columns = joblib.load('feature_columns.pkl')\n
try:\n    scaler = joblib.load('scaler.pkl')\n    usa_scaler = True\nexcept:\n    scaler = None\n    usa_scaler = False\n
def prever_churn_cliente(cliente_dict):\n    """Prevê probabilidade de churn de um cliente."""\n    
    df_novo = pd.DataFrame([cliente_dict])\n    df_encoded = pd.get_dummies(df_novo, drop_first=True)\n    
    for col in feature_columns:\n        if col not in df_encoded.columns:\n            df_encoded[col] = 0\n    df_encoded = df_encoded[feature_columns]\n    
    if usa_scaler:\n        df_encoded = scaler.transform(df_encoded)\n    
    classe = modelo.predict(df_encoded)[0]\n    prob = modelo.predict_proba(df_encoded)[0][1]\n    
    return {\n        'churn_previsto': classe,\n        'probabilidade': prob,\n        'risco': 'ALTO' if prob >= 0.7 else 'MÉDIO' if prob >= 0.4 else 'BAIXO'\n    }\n
if __name__ == '__main__':\n    # Teste\n    teste = {\n        'tenure': 12, 'MonthlyCharges': 70.0, 'TotalCharges': 840.0,\n        'Contract': 'Month-to-month', 'InternetService': 'DSL',\n        'PaymentMethod': 'Electronic check', 'OnlineSecurity': 'No',\n        'TechSupport': 'No', 'PaperlessBilling': 'Yes', 'SeniorCitizen': 0\n    }\n    print(prever_churn_cliente(teste))\n
'''

# Salvar
with open('predicao_churn.py', 'w', encoding='utf-8') as f:
    f.write(codigo_producao)

print("Arquivo criado: predicao_churn.py")
print("\nPara usar em produção:")
print("   1. Copie os arquivos: modelo_final.pkl, feature_columns.pkl, scaler.pkl (se existir)")
print("   2. Importe: from predicao_churn import prever_churn_cliente")
print("   3. Use: resultado = prever_churn_cliente(dados_cliente)")

## Resumo do Deploy

### O que foi demonstrado:

1. **Carregamento do modelo:** Como carregar o modelo salvo e artefatos necessários
2. **Função de predição:** Criação de função reutilizável para predições
3. **Interpretação:** Como interpretar probabilidades e gerar recomendações
4. **Análise em lote:** Processar múltiplos clientes de uma vez
5. **Simulação:** Testar impacto de intervenções no risco
6. **Produção:** Exportar código para uso em sistemas reais

### Próximos Passos para Produção:

#### Curto Prazo (1-3 meses):
1. **Integrar com CRM:** 
   - Conectar com Salesforce/HubSpot/sistema interno
   - Alertas automáticos para clientes de alto risco

2. **Dashboard de Monitoramento:**
   - Visualização em tempo real de clientes em risco
   - Métricas de performance do modelo

3. **API REST:**
   ```python
   # Flask/FastAPI
   @app.post('/prever-churn')
   def api_prever(cliente: ClienteSchema):
       return prever_churn_cliente(cliente.dict())
   ```

#### Médio Prazo (3-6 meses):
1. **Pipeline Automático:**
   - Retreinamento mensal com novos dados
   - Validação automática de performance
   - Versionamento de modelos

2. **A/B Testing:**
   - Testar diferentes estratégias de retenção
   - Medir ROI real das intervenções

3. **Expansão de Features:**
   - Dados comportamentais (uso de dados, chamadas)
   - Histórico de reclamações
   - Interações com atendimento

#### Longo Prazo (6-12 meses):
1. **Modelos Avançados:**
   - XGBoost, LightGBM
   - Neural Networks
   - Ensemble methods

2. **Sistema de Recomendação:**
   - Ações personalizadas por cliente
   - Otimização do custo de retenção

3. **Expansão:**
   - Outros produtos/serviços da empresa
   - Previsão de upsell/cross-sell

### Considerações Finais:

- **Monitoramento contínuo:** Performance do modelo pode degradar com o tempo
- **Feedback loop:** Coletar resultados das intervenções para melhorar o modelo
- **Ética:** Transparência com clientes sobre uso de predições
- **Compliance:** LGPD e proteção de dados pessoais

---

**O modelo está pronto para uso!**